<a href="https://colab.research.google.com/github/Waqasii/BestFeatureSelectionUsingGA/blob/main/FeatureSelectionUsingGA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Necessary Imports

In [5]:
import time
import random
import pandas as pd
import numpy as np
import copy
import math
import tensorflow
import sklearn

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

from sklearn.metrics import accuracy_score

Load the data

In [6]:
dataframe = pd.read_csv('bank-additional-full.csv', sep=";")

print(dataframe.shape)
print(dataframe.columns)
# dataframe = pd.get_dummies(dataframe)
# print(dataframe.shape)
# print(dataframe.columns)

(41188, 21)
Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx',
       'cons.conf.idx', 'euribor3m', 'nr.employed', 'y'],
      dtype='object')


Add 5 dummy colm 

In [7]:
# Generate random values for each dummy column
dummy1 = np.random.rand(dataframe.shape[0])
dummy2 = np.random.rand(dataframe.shape[0])
dummy3 = np.random.rand(dataframe.shape[0])
dummy4 = np.random.rand(dataframe.shape[0])
dummy5 = np.random.rand(dataframe.shape[0])

# Create a DataFrame with the dummy columns
dummy_df = pd.DataFrame({
    'dummy1': dummy1,
    'dummy2': dummy2,
    'dummy3': dummy3,
    'dummy4': dummy4,
    'dummy5': dummy5
})

# Concatenate the original DataFrame with the dummy DataFrame
dataframe = pd.concat([dataframe, dummy_df], axis=1)

In [8]:
def convert_to_numeric(dataframe):
    # Loop over all columns in the dataframe
    for col_name in dataframe.columns:
        # Check if the column is categorical
        if dataframe[col_name].dtype.name == 'category':
            # Convert the categorical column to numeric using cat.codes
            dataframe[col_name] = dataframe[col_name].cat.codes
        # Check if the column is of type 'object'
        elif dataframe[col_name].dtype.name == 'object':
            # Convert the 'object' column to numeric using factorize
            dataframe[col_name] = pd.factorize(dataframe[col_name])[0]
    return dataframe


dataframe = convert_to_numeric(dataframe)

# GETTER FOR UNIQUE VALUES
def get_uniques(dataframe):
    for feature in dataframe.columns:

        print(f"Feature {feature}: \n{list(pd.Series(dataframe[feature]).unique())}")

get_uniques(dataframe)



Feature age: 
[56, 57, 37, 40, 45, 59, 41, 24, 25, 29, 35, 54, 46, 50, 39, 30, 55, 49, 34, 52, 58, 32, 38, 44, 42, 60, 53, 47, 51, 48, 33, 31, 43, 36, 28, 27, 26, 22, 23, 20, 21, 61, 19, 18, 70, 66, 76, 67, 73, 88, 95, 77, 68, 75, 63, 80, 62, 65, 72, 82, 64, 71, 69, 78, 85, 79, 83, 81, 74, 17, 87, 91, 86, 98, 94, 84, 92, 89]
Feature job: 
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
Feature marital: 
[0, 1, 2, 3]
Feature education: 
[0, 1, 2, 3, 4, 5, 6, 7]
Feature default: 
[0, 1, 2]
Feature housing: 
[0, 1, 2]
Feature loan: 
[0, 1, 2]
Feature contact: 
[0, 1]
Feature month: 
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Feature day_of_week: 
[0, 1, 2, 3, 4]
Feature duration: 
[261, 149, 226, 151, 307, 198, 139, 217, 380, 50, 55, 222, 137, 293, 146, 174, 312, 440, 353, 195, 38, 262, 342, 181, 172, 99, 93, 233, 255, 362, 348, 386, 73, 230, 208, 336, 365, 1666, 577, 366, 314, 160, 212, 188, 22, 616, 178, 355, 225, 266, 253, 179, 269, 135, 161, 787, 145, 449, 812, 164, 357, 232, 91, 273, 158, 177, 200, 176, 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [9]:
# GETTER FOR MISSING VALUES
def get_missing_values(dataframe):
    for col in dataframe.isnull():
        all = list()
        print(f'Feature {col}:\n\n')
        for index, instance in enumerate(dataframe.isnull()[col]):
            if instance is True:
                all.append(index)
        print(f'Missing instances: {all}')
        print('Number of missing instances: ', len(all))

get_missing_values(dataframe)

Feature age:


Missing instances: []
Number of missing instances:  0
Feature job:


Missing instances: []
Number of missing instances:  0
Feature marital:


Missing instances: []
Number of missing instances:  0
Feature education:


Missing instances: []
Number of missing instances:  0
Feature default:


Missing instances: []
Number of missing instances:  0
Feature housing:


Missing instances: []
Number of missing instances:  0
Feature loan:


Missing instances: []
Number of missing instances:  0
Feature contact:


Missing instances: []
Number of missing instances:  0
Feature month:


Missing instances: []
Number of missing instances:  0
Feature day_of_week:


Missing instances: []
Number of missing instances:  0
Feature duration:


Missing instances: []
Number of missing instances:  0
Feature campaign:


Missing instances: []
Number of missing instances:  0
Feature pdays:


Missing instances: []
Number of missing instances:  0
Feature previous:


Missing instances: []
Number of missin

Define functions for calculating metrics

In [10]:
from keras import backend as K

def recall_fun(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_fun(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_fun(y_true, y_pred):
    precision = precision_fun(y_true, y_pred)
    recall = recall_fun(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

Define functions to be used for the Genetic Algorithm

In [11]:
def generate_dataframes_for_training(dataframe, split_frac=0.8):
    """
    Generates training and testing dataframes from a complete dataframe, according to the split_frac parameter
    """
    
    train_data = dataframe.sample(frac=split_frac, random_state=314)
    test_data = dataframe.drop(train_data.index)
    
    X_train = train_data.loc[:, train_data.columns != 'y']
    X_test = test_data.loc[:, test_data.columns != 'y']
    y_train = train_data.loc[:, train_data.columns == 'y']
    y_test = test_data.loc[:, test_data.columns == 'y']

    print(X_train.shape)
    print(X_test.shape)
    print(y_train.shape)
    print(y_test.shape)
    
    return X_train, X_test, y_train, y_test

In [12]:
def get_n_individual(counter, population):
    """
    If counter is 0, return the individual with the highest prob
    If counter is 1, return the second individual with the highest prob
    If counter is 2, return the third individual withthe highest prob
    """
    index = counter + 1
    probabilities = [ind[1] for ind in population]
    sorted_probs = sorted(probabilities, key=float)
    max_prob = probabilities[-index]
    max_individual = [ind[0] for ind in population if ind[1] == max_prob][0]
    
    return max_individual

In [13]:
def generate_random_individuals(num_individuals, num_features, max_features=None, verbose=False):
    """
    Randomly generates individuals

    The number of individuals to generate is given by the num_individuals parameter
    The length of each individual is equal to the num_features parameter
    The maximum number of active features for every individual is given by the max_features parameter
    """
    if verbose: print('GENERATING RANDOM INDIVIDUALS.... ')
        
    individuals = list()
    
    for _ in range(num_individuals):
        individual = ''
        for col in range(num_features):
            # For each char in the individual, a 1 or a 0 is randomly generated
            if individual.count('1') == max_features:
                individual += '0'
                continue
                
            individual += str(random.randint(0, 1))
            
        if verbose: print(f'Genrated a new indivudal: {individual}')
        individuals.append(individual)
        
    if verbose: print(f'Generated list of {num_individuals} individuals: {individuals}')
        
    return individuals


# individuals_temp = generate_random_individuals(10, 20, 10)
# print(individuals_temp)

In [26]:
def create_model(X, verbose=False):
    """
    X: training dataset to be used. Its shape is used to set the input shape for the model.
    """
    
    model = Sequential()
    
    model.add(Dense(32, input_dim=X.shape[1], activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    if verbose: print('MODEL SUMMARY: \n')
    if verbose: print(model.summary())
    
    model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    
    return model



def get_weights(population):
    """
    Calculate weights from the population filled with the accuracies
    """
    total_accuracies = 0
    new_population = []
    
    # Get the sum of all accuracies of the population
    for individual in population:
        total_accuracies += individual[1]
        
    # For each individual, calculate its weight by dividing its accuracy by the overall sum calculated above
    for individual in population:
        weight = individual[1]/total_accuracies
        # Store the individual and its weight in the final population list
        new_population.append((individual[0], float(weight*100)))
        
    return new_population



def get_fitness_func(individual, dataframe, verbose=False):
    """
    Calculate accuracy for the individual passed as parameter.
    Both the dataframe and the y_data parameters are used for training and evaluating the model.
    """
    if verbose: print('Calculating accuracy for individual ', individual)
    
    # generate_dataframes_for_training() function splits a given dataset into training and test data, 
    # and separates labels and rest of features
    X_train, X_test, y_train, y_test = generate_dataframes_for_training(dataframe)
    
    X_train = X_train.loc[:, [True if individual[i] == '1' else False for i in range(len(individual))]]
    X_test = X_test.loc[:, [True if individual[i] == '1' else False for i in range(len(individual))]]    
    
    model = create_model(X_train, individual)
    
    X_train = np.asarray(X_train).astype(np.float64)
    X_test = np.asarray(X_test).astype(np.float64)
    y_train = np.asarray(y_train).astype(np.float64)
    y_test = np.asarray(y_test).astype(np.float64)
    
    model.fit(X_train, y_train, epochs=100, verbose=1 if verbose else 0)
    
    pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, pred.round())
    if verbose: print(f"Accuracy for the classifier trained for individual {individual}: ", accuracy)
    
    return accuracy



def fill_population(individuals, dataframe, verbose=False):
    """
    Fills the population list with individuals and their weights
    """
    population = list()
    
    for individual in individuals:
        
        # Get the value of the fitness function (accuracy of the model)
        if verbose: print(f'Calculating fitness function value for individual {individual}')
        accuracy = get_fitness_func(individual, dataframe, verbose)
        
        # Check that the value is not the goal state (in this case, an accuracy of 80% is a terminal state)
        if float(accuracy) > 0.95:
            if verbose: print(f'Goal state found for individual {individual}')
            return individual
            
        individual_complete = (individual, accuracy)
        population.append(individual_complete)
        
    # The final population list is created, which contains each individual together with its weight
    # (weights will be used in the reproduction step)
    new_population = get_weights(population)
    if verbose: print(f'Generated population list (with weights): {new_population}')
    
    return new_population



def choose_parents(population, counter):
    """
    From the population, weighting the probabilities of an individual being chosen via the fitness
    function, takes randomly two individual to reproduce
    Population is a list of tuples, where the first element is the individual and the second
    one is the probability associated to it.
    To avoid generating repeated individuals, 'counter' parameter is used to pick parents in different ways, thus
    generating different individuals
    """
    # Pick random parent Number 1 and Number 2
    # (get_n_individual() function randomly picks an individual following the distribution of the weights)
    if counter == 0:        
        parent_1 = get_n_individual(0, population)        
        parent_2 = get_n_individual(1, population)
    elif counter == 1:
        parent_1 = get_n_individual(0, population)        
        parent_2 = get_n_individual(2, population)
        
    else:
        probabilities = (individual[1] for individual in population)
        individuals = [individual[0] for individual in population]
        parent_1, parent_2 = random.choices(individuals, weights=probabilities, k=2)
    
    return [parent_1, parent_2]


  
def mutate(child, prob=0.1):
    """
    Randomly mutates an individual according to the probability given by prob parameter
    """
    new_child = copy.deepcopy(child)
    for i, char in enumerate(new_child):
        if random.random() < prob:
            new_value = '1' if char == '0' else '0'
            new_child = new_child[:i] + new_value + new_child[i+1:]
    
    return new_child


  
def reproduce(individual_1, individual_2):
    """
    Takes 2 individuals, and combines their information based on a
    randomly chosen crosspoint.
    Each reproduction returns 2 new individuals
    """ 
    # Randomly generate a integer between 1 and the length of the individuals minus one, which will be the crosspoint
    crosspoint = random.randint(1, len(individual_1)-1)
    child_1 = individual_1[:crosspoint] + individual_2[crosspoint:]
    child_2 = individual_2[:crosspoint] + individual_1[crosspoint:]
    child_1, child_2 = mutate(child_1), mutate(child_2)
 
    return [child_1, child_2]


  
def generation_ahead(population, verbose=False):
    """
    Reproduces all the steps for choosing parents and making 
    childs, which means creating a new generation to iterate with
    """
    new_population = list()
    
    for _ in range(int(len(population)//2)):      
        # According to the weights calculated before, choose a set of parents to reproduce
        parents = choose_parents(population, counter=_)
        if verbose: print(f'Parents chosen: {parents}')
          
        # Reproduce the pair of individuals chose above to generate two new individuals
        childs = reproduce(parents[0], parents[1])
        if verbose: print(f'Generated children: {childs}\n')
        new_population += childs
        
    return new_population



def main_loop(ind_num, dataframe, max_iter=5, verbose=False):
    """
    Performs all the steps of the Genetic Algorithm
    1. Generate random population
    2. Fill population with the weights of each individual
    3. Check if the goal state is reached
    4. Reproduce the population, and create a new generation
    5. Repeat process until termination condition is met
    """
    print('Number of features:', dataframe.shape[1] )
    # Generate individuals (returns a list of strings, where each str represents an individual)
    individuals = generate_random_individuals(ind_num, dataframe.shape[1]-1, 12, verbose)
    
    # Returns a list of tuples, where each tuple represents an individual and its weight
    population = fill_population(individuals, dataframe, verbose)
    
    # Check if a goal state is reached
    # When goal state is reached, fill_population() function returns a str, otherwise continue
    if isinstance(population, str):
        return population
        
    # Reproduce current generation to generate a better new one
    new_generation = generation_ahead(population, verbose)
    
    # After the new generation is generated, the loop goes on until a solution is found or until the maximum number of
    # iterations are reached
    iteration_count = 0
    while iteration_count < max_iter:
        if verbose: print(f'\n\n\nITERATION NUMBER {iteration_count+1} (Iteration max = {max_iter+1})\n\n\n')
        population = fill_population(new_generation, dataframe, verbose)
        
        # Check if a goal state is reached
        if isinstance(population, str):
            break
        
        new_generation = generation_ahead(population, verbose)   
        iteration_count += 1
        
    return population


Create and train the model with the complete dataset


In [15]:
# Create the model
model = Sequential()
model.add(Dense(64, input_dim=dataframe.shape[1]-1, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
  
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [16]:
# Generate dataframes for both training and testing
X_train, X_test, y_train, y_test = generate_dataframes_for_training(dataframe)


# test
# # Convert categorical variables to numerical variables using one-hot encoding
# X_train = pd.get_dummies(X_train)
# X_test = pd.get_dummies(X_test)
# y_train = pd.get_dummies(y_train)
# y_test = pd.get_dummies(y_test)


X_train = np.asarray(X_train).astype('float32')
y_train = np.asarray(y_train).astype('float32')
X_test = np.asarray(X_test).astype('float32')
y_test = np.asarray(y_test).astype('float32')

(32950, 25)
(8238, 25)
(32950, 1)
(8238, 1)


In [17]:
start = time.time()
print(X_train)
model.fit(X_train, y_train, epochs=128, verbose=1)
print('Time elapsed for training the model with the full dataset: ', time.time() - start, ' seconds')


[[35.          4.          0.         ...  0.9855555   0.6848991
   0.34664458]
 [32.          4.          1.         ...  0.9788969   0.06341633
   0.8620606 ]
 [37.         10.          0.         ...  0.51322246  0.5251356
   0.15517113]
 ...
 [45.          9.          0.         ...  0.45744115  0.791472
   0.5553171 ]
 [41.          4.          0.         ...  0.6606085   0.7652382
   0.6692738 ]
 [29.          3.          0.         ...  0.47079158  0.8170754
   0.36550188]]
Epoch 1/128
1030/1030 [==============================] - 3s 2ms/step - loss: 5.4794 - accuracy: 0.8429
Epoch 2/128
1030/1030 [==============================] - 2s 2ms/step - loss: 3.2810 - accuracy: 0.8510
Epoch 3/128
1030/1030 [==============================] - 2s 2ms/step - loss: 2.1496 - accuracy: 0.8558
Epoch 4/128
1030/1030 [==============================] - 2s 2ms/step - loss: 1.3181 - accuracy: 0.8576
Epoch 5/128
1030/1030 [==============================] - 2s 2ms/step - loss: 0.8601 - accuracy: 0.8619

In [18]:
print('Recall: ', recall_fun(y_test, pred.round()))
print('Precision: ', precision_fun(y_test, pred.round()))
print('F1 Score: ', f1_fun(y_test, pred.round()))

NameError: ignored

Create and train the model with the optimized dataset

First, the relevant features must be selected by applying the GA

In [ ]:
start = time.time()
final_population = main_loop(100, dataframe, verbose=True)
print('Time elapsed for executing the recursive GA: ', time.time() - start, ' seconds')

Number of features: 26
GENERATING RANDOM INDIVIDUALS.... 
Genrated a new indivudal: 0011100001110001110011100
Genrated a new indivudal: 1110110000010100100111010
Genrated a new indivudal: 1001011010101000001001011
Genrated a new indivudal: 1100000010110001100000100
Genrated a new indivudal: 1110101000011101111000000
Genrated a new indivudal: 1101001110101111100000000
Genrated a new indivudal: 0011010100000110101111000
Genrated a new indivudal: 0100000101010010001001110
Genrated a new indivudal: 0000111011100000110110101
Genrated a new indivudal: 1101011011110010100100000
Genrated a new indivudal: 0011000110101001110000001
Genrated a new indivudal: 1111010010101011100100000
Genrated a new indivudal: 0011001100011010110100110
Genrated a new indivudal: 0001001100000110111010110
Genrated a new indivudal: 1000111110000110001001110
Genrated a new indivudal: 0111101000111101011000000
Genrated a new indivudal: 1001101001100101100110100
Genrated a new indivudal: 0001110110110000000000000
Genrat

In [ ]:
# retrieve the individual with highest weight
print((final_population))
# max = -99
# for i in range(len(final_population)):
#     if final_population[i][1] > max:
#         max = final_population[i][1]
#         max_ind = final_population[i][0]

max = final_population[1]
max_ind = final_population[0]


print('Total selcted Features:', str(final_population.count('1')))

Finally, leave only the relevant features and train the model

In [ ]:
# print(dataframe)
# print(optimized_dataframe)
# print(max_ind)
# print(max)
optimized_dataframe = dataframe.loc[:, [True if char == '1' else False for char in final_population+'0']]
optimized_dataframe['y'] = dataframe['y']


print(optimized_dataframe)

In [22]:
# Create the model
model = Sequential()
model.add(Dense(64, input_dim=optimized_dataframe.shape[1]-1, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
  
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [23]:
# Generate dataframes for both training and testing
X_train, X_test, y_train, y_test = generate_dataframes_for_training(optimized_dataframe)
X_train = np.asarray(X_train).astype('float32')
X_test = np.asarray(X_test).astype('float32')
y_train = np.asarray(y_train).astype('float32')
y_test = np.asarray(y_test).astype('float32')

(32950, 12)
(8238, 12)
(32950, 1)
(8238, 1)


In [24]:
start = time.time()
model.fit(X_train, y_train, epochs=128, verbose=1)
print('Time elapsed for training the model with the full dataset: ', time.time() - start, ' seconds')

Epoch 1/128
1030/1030 [==============================] - 3s 2ms/step - loss: 6.5445 - accuracy: 0.7863
Epoch 2/128
1030/1030 [==============================] - 2s 2ms/step - loss: 4.0251 - accuracy: 0.7915
Epoch 3/128
1030/1030 [==============================] - 2s 2ms/step - loss: 2.8045 - accuracy: 0.7929
Epoch 4/128
1030/1030 [==============================] - 2s 2ms/step - loss: 2.1166 - accuracy: 0.7993
Epoch 5/128
1030/1030 [==============================] - 2s 2ms/step - loss: 1.4625 - accuracy: 0.8036
Epoch 6/128
1030/1030 [==============================] - 2s 2ms/step - loss: 0.9805 - accuracy: 0.8188
Epoch 7/128
1030/1030 [==============================] - 2s 2ms/step - loss: 0.6551 - accuracy: 0.8363
Epoch 8/128
1030/1030 [==============================] - 2s 2ms/step - loss: 0.5098 - accuracy: 0.8465
Epoch 9/128
1030/1030 [==============================] - 2s 2ms/step - loss: 0.4220 - accuracy: 0.8632
Epoch 10/128
1030/1030 [==============================] - 2s 2ms/step - l

In [25]:
print('Recall: ', recall_fun(y_test, pred.round()))
print('Precision: ', precision_fun(y_test, pred.round()))
print('F1 Score: ', f1_fun(y_test, pred.round()))

NameError: ignored